## Homework 1

In [11]:
#Q1 Group by and aggregate statistics
import pandas as pd

sales_data = pd.read_csv('sales_data.csv') #loading data from csv file 

grouped = sales_data.groupby('Category')
total_quantity = grouped['Quantity'].sum()  # Total quantity sold per category
average_price = grouped['Price'].mean()     # Average price per unit per category
max_quantity = grouped['Quantity'].max()    # Maximum quantity in a single transaction per category

category_stats = pd.DataFrame({
    'Total Quantity': total_quantity,
    'Average Price': average_price,
    'Max Quantity': max_quantity
})
print("Category Statistics:") #printing combined statistics 
print(category_stats)

Category Statistics:
             Total Quantity  Average Price  Max Quantity
Category                                                
Clothing                157      31.176471            15
Electronics             183     276.764706            15
Home                    144      55.000000            14


In [12]:
#Q2 Top-selling product 
product_sales = sales_data.groupby(['Category', 'Product'])['Quantity'].sum().reset_index()
top_products = product_sales.loc[product_sales.groupby('Category')['Quantity'].idxmax()]
print("\nTop-Selling Products by Category:")
print(top_products)



Top-Selling Products by Category:
       Category          Product  Quantity
8      Clothing            Jeans        15
27  Electronics         Smart TV        15
46         Home  Pressure Cooker        14


In [ ]:
#Q3 Highest total sale date 
sales_data['Total Sales'] = sales_data['Quantity'] * sales_data['Price']  # Calculate total sales per transaction
daily_sales = sales_data.groupby('Date')['Total Sales'].sum()  # Sum total sales by date
max_sales_date = daily_sales.idxmax()   # Date with highest total sales
max_sales_value = daily_sales.max()   # Value of highest total sales
print(f"\nThe date with the highest total sales is {max_sales_date} with total sales of ${max_sales_value:.2f}")


The date with the highest total sales is 2023-01-07 with total sales of $15150.00


## Homework 2

In [22]:
#Q1 
import pandas as pd

df = pd.read_csv('customer_orders.csv') #loading data from file 

order_counts = df.groupby('CustomerID')['OrderID'].count() #finding customers with 20+ orders
frequent_customers = order_counts[order_counts >= 20].index
print("Customers with 20 or more orders:", frequent_customers.tolist())

Customers with 20 or more orders: [101, 102, 103, 104]


In [24]:
#Q2 
average_prices = df.groupby('CustomerID')['Price'].mean() #customers with avg price > 120 
high_avg_customers = average_prices[average_prices > 120].index
print("Customers with average price > $120:", high_avg_customers.tolist())

Customers with average price > $120: [102, 104]


In [25]:
#Q3 
product_summary = df.groupby('Product').apply(lambda x: pd.Series({
    'total_quantity': x['Quantity'].sum(),
    'total_price': (x['Quantity'] * x['Price']).sum()
}))
filtered_products = product_summary[product_summary['total_quantity'] >= 5]
print("Products with total quantity >= 5 units:")
print(filtered_products)

Products with total quantity >= 5 units:
                  total_quantity  total_price
Product                                      
Cargo Pants                    6          180
Dress Shirt                    5          125
Formal Shirt                   6          210
Smartphone                     5         2000
Sport Shoes                    5          200
Sunglasses                     5           75
Wireless Earbuds               6          720


C:\Users\ITPARK\AppData\Local\Temp\ipykernel_17940\1339068874.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  product_summary = df.groupby('Product').apply(lambda x: pd.Series({


## Homework 3

In [ ]:
import sqlite3
import pandas as pd

# Connecting to the database
conn = sqlite3.connect('population.db')

# Getting data from the population table
df = pd.read_sql_query("SELECT salary, state FROM population", conn)

# Closing the database
conn.close()

salary_bands = [
    (0, 200000), (200001, 400000), (400001, 600000), (600001, 800000),
    (800001, 1000000), (1000001, 1200000), (1200001, 1400000), (1400001, 1600000),
    (1600001, 1800000), (1800001, float('inf'))
]

# Storing results for salary bands
band_results = {}
for low, high in salary_bands:
    band = f"{low:,}-{high:,}" if high != float('inf') else f"{low:,} and over"
    band_results[band] = {'count': 0, 'sum': 0, 'salaries': []}

# Storing results for states
state_results = {}

# Categorizin data
for _, row in df.iterrows():
    salary = row['salary']
    state = row['state']
    
    # Finding salary band
    for (low, high) in salary_bands:
        if low <= salary < high:
            band = f"{low:,}-{high:,}" if high != float('inf') else f"{low:,} and over"
            band_results[band]['count'] += 1
            band_results[band]['sum'] += salary
            band_results[band]['salaries'].append(salary)
            break
    
    # Adding to state data
    if state in state_results:
        state_results[state]['count'] += 1
        state_results[state]['sum'] += salary
        state_results[state]['salaries'].append(salary)
    else:
        state_results[state] = {'count': 1, 'sum': salary, 'salaries': [salary]}

# Calculating measures
band_data = []
for band, data in band_results.items():
    if data['count'] > 0:
        pct = (data['count'] / len(df)) * 100
        avg = data['sum'] / data['count']
        med = pd.Series(data['salaries']).median()
        band_data.append([band, f"{pct:.2f}%", f"${avg:,.2f}", f"${med:,.2f}", data['count']])

state_data = []
for state, data in state_results.items():
    if data['count'] > 0:
        pct = (data['count'] / len(df)) * 100
        avg = data['sum'] / data['count']
        med = pd.Series(data['salaries']).median()
        state_data.append([state, f"{pct:.2f}%", f"${avg:,.2f}", f"${med:,.2f}", data['count']])

# Creating DataFrames
band_df = pd.DataFrame(band_data, columns=['Salary Band', 'Percentage', 'Average Salary', 'Median Salary', 'Number of population'])
state_df = pd.DataFrame(state_data, columns=['State', 'Percentage', 'Average Salary', 'Median Salary', 'Number of population'])

# Printing results
print("Salary Band Analysis:")
print(band_df)
print("\nState Analysis:")
print(state_df)

# Saving to Excel
with pd.ExcelWriter('population_salary_analysis_results.xlsx') as writer:
    band_df.to_excel(writer, sheet_name='Salary Bands', index=False)
    state_df.to_excel(writer, sheet_name='States', index=False)